In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from typing import *

from tqdm import tqdm

import pickle
import multiprocessing
import collections

import lib
import workers
import word_bank

## Best starting word

Numbers indicate how many possibilities will be remaining after guessing the word.

Considering all words (12972):
- SERAI (697 worst, 315 avg)
- SOARE (769 worst, 304 avg)

Considering only answers (2315):
- AESIR (168 worst, 62 avg)
- REAIS (168 worst, 52 avg)
- RAISE (168 worst, 52 avg)

## Best following words from RAISE

 1. `_ _ _ _ _ `: NOBLY (15 worst)
 2. `_ _ _ _ e?`: TOWEL (10 worst)
 3. `_ _ i?_ _ `: UNTIL (11 worst)
 4. `r?_ _ _ _ `: COLON (10 worst)
 5. `r?_ _ _ e?`: OUTER (16 worst)
 6. `_ a?_ _ _ `: CLOAK (7 worst)
 7. `_ a!_ _ _ `: TANGY (13 worst)
 8. `_ _ _ s?_ `: STOOL (8 worst)
 9. `r?a?_ _ _ `: CROWN (10 worst)
10. `_ a?_ _ e?`: METAL (5 worst)
11. `_ _ _ _ e!`: CLOUT (5 worst)
12. `_ _ i!_ _ `: CLOUT (5 worst)
13. `_ a?_ s?_ `: STALK (5 worst)
14. `_ a?_ _ e!`: ALBUM (7 worst)
15. `_ _ _ s?e?`: SPELT (4 worst)
16. `r?_ _ _ e!`: PRONG (6 worst)
17. `_ _ i?_ e?`: LIPID (3 worst)
18. `_ a?i?_ _ `: PLANT (3 worst)
19. `r?a?_ _ e?`: DRAFT (5 worst)
20. `_ _ i!s?_ `: BLUNT (4 worst)
21. `r?a!_ _ e?`: EMPTY (7 worst)
22. `r?_ i!_ _ `: PLANK (5 worst)
23. `r?a?_ _ e!`: GRACE (6 worst)
24. `_ a!_ _ e!`: GULLY (4 worst)
25. `r?a!_ _ _ `: AMPLY (5 worst)
26. `r?_ i?_ e?`: FIEND (6 worst)
27. `_ _ i?_ e!`: GULCH (4 worst) 
28. `r?_ _ s?_ `: BURNT (3 worst)
29. `r?_ i?_ _ `: CHOIR (3 worst)
30. `_ _ i!_ e!`: CLOTH (3 worst)

Or when in doubt, guess **CLOUT** (22 worst, 5.1 average)

In [ ]:
for second_guess in [
  'nobly',
  'towel',
  'until',
  'colon',
  'outer',
  'cloak',
  'tangy',
  'stool',
  'crown',
  'metal',
  'clout',
  'clout',
  'stalk',
  'album',
  'spelt',
  'prong',
  'lipid',
  'plant',
  'draft',
  'blunt',
  'empty',
  'plank',
  'grace',
  'gully',
  'amply',
  'fiend',
  'gulch',
  'burnt',
  'choir',
  'cloth',
]:
  posss = []
  for answer in tqdm(word_bank.answers):
    clues = lib.get_clues('serai', answer) + lib.get_clues(second_guess, answer)
    poss = word_bank.get_possibilities_num(clues, word_bank.answers)
    posss.append(poss)
  print(second_guess, max(posss), sum(posss)/len(posss))


In [ ]:
clue_freq = collections.defaultdict(int)
for word in word_bank.answers:
  clues = lib.get_clues('serai', word)
  clue_freq[clues] += 1

items = sorted(clue_freq.items(), key=lambda x: x[1], reverse=True)

for clues, _ in items[:1]:
  print(clues)
  print(workers.play(clues, word_bank.answers))

In [ ]:
clues = lib.get_clues('raise', 'knoll') + lib.get_clues('nobly', 'knoll')
workers.play(clues, word_bank.answers, True)

In [ ]:
def play_game(answer: str, hard_mode=True):
  guesses = ['raise']
  clues = lib.get_clues(guesses[0], answer)

  while guesses[-1] != answer:
    guess, max_poss = workers.play(clues, word_bank.answers, hard_mode)
    guesses += [guess]
    clues += lib.get_clues(guess, answer)
  
  return guesses

In [ ]:
performance = collections.defaultdict(int)
games = []
for answer in tqdm(word_bank.answers):
  guesses = play_game(answer)
  games.append(guesses)
  performance[len(guesses)] += 1
performance

In [ ]:
with open('optimal-games-hard-mode.pkl', 'wb') as file:
  pickle.dump(games, file)

In [ ]:
for game in games:
  if len(game) > 6:
    print(game)
    print(play_game(game[-1], False))

In [ ]:
clues = lib.get_clues('raise', 'batch') + lib.get_clues('tangy', 'batch') + lib.get_clues('clump', 'batch') + lib.get_clues('bawdy', 'batch')
workers.play(clues, word_bank.answers, False)

In [ ]:
with open('max-possibilities-all-words.pkl', 'rb') as file:
  max_poss_all_words = pickle.load(file)

In [ ]:
clues = sum(
  (
    lib.get_clues(word, goal)
    for word, goal in [
      ('raise', 'aXXXX'),
      ('cloak', 'caXXX'),
    ]
  ),
  tuple()
)
workers.play(clues, word_bank.answers)

In [ ]:
best = None
results = []

with multiprocessing.Pool() as pool:
  input_words = [word for _, word in sorted(max_poss_all_words)]
  for result in tqdm(pool.imap_unordered(workers.get_worst_case_with_answers, input_words), total=len(words)):
    results.append(result)
    if best is None or result < best:
      best = result
      print(best)


In [ ]:
with open('max-possibilities-only-answers.pkl', 'wb') as file:
  pickle.dump(results, file)

In [ ]:
sum(lib.get_possibilities_num(lib.get_clues('aesir', word)) for word in lib.words)/len(lib.words)

In [ ]:
for _, guess in sorted(results)[:10]:
  print(guess, sum(lib.get_possibilities_num(lib.get_clues(guess, word)) for word in lib.words)/len(lib.words))